<a href="https://colab.research.google.com/github/Hritik1100/Depression-Prediction-using-ML/blob/main/Depression_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Depression Prediction on Twitter using Machine Learning Algorithms**

Import data and download required libraries

In [ ]:
import pandas as pd
import re
import nltk
import sklearn
import spacy
data = pd.read_csv('newfile.csv',encoding='latin-1')

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

In [ ]:
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
import string
lem = WordNetLemmatizer()
corpus_test = []
stopwords = set(stopwords.words('english'))
punc = string.punctuation

In [ ]:
!pip install git+https://github.com/casics/spiral.git
!pip install textaugment
!pip install contractions
!pip install -U git+https://github.com/ray-project/tune-sklearn.git && pip install 'ray[tune]'
!!pip install scikit-optimize

In [ ]:
from textaugment import Wordnet
t = Wordnet()
aug = t.augment("The quick brown fox jumps over the lazy dog")
print(aug)

the quick brown fox skip over the lazy dog


Preprocessing

In [ ]:
corpus=[]
import contractions
from spiral import ronin
for i in range(len(data)):
  #review = re.sub(r"http\S+", "", data['Tweets'][i])
  #review = re.sub('[^a-zA-Z]',' ',review)
  review = re.sub(r"http\S+", "", data['message'][i]) # remove urls
  review = re.sub(r'<([^>]*)>', ' ', review) # remove emojis
  review = re.sub(r'@\w+', ' ', review) # remove at mentions
  review = re.sub(r'#', '', review) # remove hashtag symbol
  review = re.sub(r'[0-9]+', ' ', review) # remove numbers
  review = re.sub(r'[^A-Za-z0-9,?.!]+', ' ', review)
  review = review.lower()
  l=[]
  for word in review.split():
    l.append(contractions.fix(word))
  review = ' '.join(l)
  review = t.augment(review)
  review = ronin.split(review)
  review = ' '.join(review)
  review = nltk.word_tokenize(review)
  review = [lem.lemmatize(word) for word in review if word not in stopwords and word not in punc]
  review = ' '.join(review)
  corpus.append(review)

Splitting data into Train and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X = corpus
y = data['label']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

Feature Engineering (Using Count Vectorizer)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
fun = CountVectorizer(max_features=1500)
X_train_vector = fun.fit_transform(X_train).toarray()
X_test_vector = fun.transform(X_test).toarray()

Hyper Parameters

In [ ]:
params = {'clf__max_depth': [200,300],
          'clf__max_features': ['sqrt'],
          'clf__min_samples_leaf': [2],
          'clf__splitter': ['best']}




Finding the right Hyperparameters using TuneSearchCV

In [ ]:
from sklearn.metrics import precision_score,confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report
from tune_sklearn import TuneSearchCV
from imblearn.over_sampling import SMOTE           #Run this for Cross Validation after initializing params
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'weighted'),
           'recall': make_scorer(recall_score, average = 'weighted'),
           'f1': make_scorer(f1_score, average = 'weighted')
}

pipeline = Pipeline([('Smote',SMOTE(random_state=42)),('clf',DecisionTreeClassifier())])
tune_imba = TuneSearchCV(pipeline,params,scoring=scoring,max_iters=10,n_jobs=-1,search_optimization = 'bayesian',cv=5,verbose=1,refit='precision')
tune_imba.fit(X_train_vector,y_train)
y_pred = tune_imba.predict(X_test_vector)
print('validation Precision',tune_imba.best_score_)
print("Test precision score - ", precision_score(y_test, y_pred))
print("Test recall score - ", recall_score(y_test, y_pred))
print("Test f1 score - ", f1_score(y_test, y_pred))
print("Test accuracy score - ", accuracy_score(y_test, y_pred))
tune_imba.best_params_

INFO:ray.tune.tune:Total run time: 385.57 seconds (385.50 seconds for the tuning loop).
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


validation Precision 0.8285910987559312
Test precision score -  0.5692599620493358
Test recall score -  0.6571741511500547
Test f1 score -  0.6100660904931366
Test accuracy score -  0.8108508014796547


{'clf__max_depth': 237,
 'clf__max_features': 'sqrt',
 'clf__min_samples_leaf': 2,
 'clf__splitter': 'best'}

Employing Gradient Boost ML Model

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

def evaluate_model(X_train, y_train, model):
    model.fit(X_train, y_train)
    preds = model.predict(X_test_vector)
    print('Accuracy:', accuracy_score(y_test, preds))
    print('F1 score:', f1_score(y_test,preds,average='weighted'))
    print('Recall:', recall_score(y_test, preds,average='weighted'))
    print('Precision:', precision_score(y_test, preds,average='weighted'))

from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=100, random_state=10)
evaluate_model(X_train_SMOTE,y_train_SMOTE,model)

Accuracy: 0.9393341553637484
F1 score: 0.9365387923693352
Recall: 0.9393341553637484
Precision: 0.9410342020495727
